In [ ]:
import openai
import os
from PyPDF2 import PdfReader

openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

In [ ]:
def get_num_pages_from_pdf(filename):
    # creating a pdf reader object
    reader = PdfReader(filename)
    return len(reader.pages)

In [ ]:
def text_from_pdf(filename, page_to_extract=-1, max_words=4097):
    # creating a pdf reader object
    reader = PdfReader(filename)
    
    text = ''
    if (page_to_extract == -1):
        for page in reader.pages:
            
            # extracting text from page
            text_tmp = page.extract_text()
            text_tmp = text_tmp.replace("\n", "")
            text += text_tmp
    else:
        if (page_to_extract > len(reader.pages)):
            return -1
        
        page = reader.pages[page_to_extract]
        # extracting text from page
        text_tmp = page.extract_text()
        text_tmp = text_tmp.replace("\n", "")
        text += text_tmp

    # Split the content into words.
    words = text.split(" ")

    # Limit the number of words.
    if len(words) > max_words:
        words = words[:max_words]

    # Return the extracted text.
    return ' '.join(words)

In [ ]:
max_dim = 1024
max_sentences = 5
temperature = 0
filename = '...'

num_pages = get_num_pages_from_pdf(filename)

for p in range(0, num_pages):
    text = text_from_pdf(filename, page_to_extract=p, max_words=max_dim)

    prompt = f"""
        List {max_sentences} important aspects ot the text delimited by triple backticks.
        ```{text}```
        """

    response = get_completion(prompt)
    print ('Page ' + str(p))
    print(response)